In [1]:
import nltk
import re
import contractions
import torchtext as text
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import numpy as np
from matplotlib import pyplot as plt

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [3]:
import spacy

nlp_sentencizer = spacy.blank("en")
nlp_sentencizer.add_pipe("sentencizer")

nlp_sentencizer.max_length = 6351665

corpus = open('Auguste_Maquet.txt', 'r')
ds = corpus.read()
ds.replace("\n", "")
tokens = nlp_sentencizer(ds)

sents = []

for sent in tokens.sents:
    sents.append(str(sent).replace('\n', " "))

In [4]:
import random

random.seed(47)
random.shuffle(sents)

train = sents[:20000]
test = sents[20001:30001]

train = ''.join(str(train))

test = ''.join(str(test))

In [5]:
def clean_text(text):
        text = text.lower()
        text = text.strip('\n')
        text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \n])|(\w+:\/\/\S+)|^rt|www.+?", " ", text)

        expanded_text = []
        for word in text.split():
            expanded_text.append(word) 
        return expanded_text

def build_vocab(text):
        vocab = {}

        for word in text:
            if word not in vocab:
                vocab[word] = len(vocab) # add a new type to the vocab
        return vocab

def process_test_data(test_data, vocab):
    process_data = test_data.copy()
    for  i, word in enumerate(test_data):
        if word not in vocab:
            process_data[i] = 'unk' 
    return process_data

            

In [6]:
train_data = clean_text(train)

test_data = clean_text(test)

vocabulary = build_vocab(train_data)
#
vocabulary['unk'] = -1

test_data_processed = process_test_data(test_data, vocabulary)